In [35]:
import nest_asyncio

nest_asyncio.apply()

In [77]:
import os
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
# 获取项目根目录路径
project_root = Path(os.getenv("PROJECT_ROOT"))
test_data_path = project_root / "readai/tests/data"

In [15]:
from llama_index.core.evaluation import (
    EmbeddingQAFinetuneDataset,
    RetrieverEvaluator,
    generate_question_context_pairs,
)

In [4]:
# from langchain_community.document_loaders import (
#     UnstructuredMarkdownLoader,
# )

# book_name = "comunication_cleaned.md"
# book_path = test_data_path / book_name
# # 使用unstructured的markdownloader加载
# loader = UnstructuredMarkdownLoader(book_path, mode="elements")
# documents = loader.load()
# # 查看节点数量
# print(f"节点数量: {len(documents)}")
# # 遍历documents,重构metadata，只保留category，filename这两个属性
# for doc in documents:
#     category = doc.metadata["category"]
#     filename = doc.metadata["filename"]
#     doc.metadata = {
#         "category": category,
#         "filename": filename,
#     }

In [ ]:
# from llama_index.core.schema import BaseNode, Document, IndexNode, NodeWithScore

# llamindex_docs = [
#     Document(
#         text=doc.page_content,
#         metadata=doc.metadata,
#         metadata_seperator="::",
#         metadata_template="{key}=>{value}",
#         text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
#     )
#     for doc in documents
# ]

In [ ]:
# from llama_index.core.node_parser import SimpleNodeParser
# from llama_index.core.schema import IndexNode

# node_parser = SimpleNodeParser.from_defaults(
#     chunk_size=1024,
#     chunk_overlap=50,
#     separator="\n\n",
#     secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
# )
# hybrid_nodes = node_parser.get_nodes_from_documents(llamindex_docs, show_progress=True)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
# sub_chunk_sizes = [128, 512, 2048]
# sub_node_parsers = [
#     SimpleNodeParser.from_defaults(
#         chunk_size=c,
#         chunk_overlap=50,
#         separator="\n\n",
#         secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
#     )
#     for c in sub_chunk_sizes
# ]

# hybrid_all_nodes = []
# for hybrid_node in hybrid_nodes:
#     for n in sub_node_parsers:
#         sub_nodes = n.get_nodes_from_documents([hybrid_node])
#         sub_inodes = [
#             IndexNode.from_text_node(sn, hybrid_node.node_id) for sn in sub_nodes
#         ]
#         hybrid_all_nodes.extend(sub_inodes)

#     # also add original node to node
#     original_node = IndexNode.from_text_node(hybrid_node, hybrid_node.node_id)
#     hybrid_all_nodes.append(original_node)

# print(len(hybrid_all_nodes))

2753


In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.deepseek import DeepSeek

api_key = os.getenv("DEEPSEEK_API_KEY")

print(api_key)
model_name = os.getenv("DEEPSEEK_MODEL")
embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5", base_url="http://localhost:11434"
)

llm = DeepSeek(
    model=model_name,
    api_key=api_key,
)

In [3]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.storage import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import AsyncQdrantClient, QdrantClient

client = QdrantClient(url="http://localhost:6333")
aclient = AsyncQdrantClient(url="http://localhost:6333")

In [5]:
collection_name = "hybrid_nodes"

In [6]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import load_index_from_storage

persist_dir = test_data_path / collection_name


In [7]:
vector_store_base = QdrantVectorStore(
    client=client,
    aclient=aclient,
    collection_name=collection_name,
)

In [ ]:
# all_nodes_docstore = SimpleDocumentStore()
# all_nodes_docstore.add_documents(hybrid_all_nodes)

In [8]:
hybrid_index = None
if client.collection_exists(collection_name):
    print(f"collection_name: '{collection_name}' 已存在")
    # 创建storage_context
    storage_context = StorageContext.from_defaults(
        persist_dir=persist_dir, vector_store=vector_store_base
    )
    hybrid_index = load_index_from_storage(
        storage_context, llm=llm, embed_model=embed_model
    )
    print("加载向量完成")
else:
    print(f"collection_name: '{collection_name}' 不存在，需要初始化")
    qdrant_storage_context = StorageContext.from_defaults(
        vector_store=vector_store_base,
        docstore=all_nodes_docstore,
    )
    hybrid_index = VectorStoreIndex(
        nodes=hybrid_all_nodes,
        embed_model=embed_model,
        llm=llm,
        storage_context=qdrant_storage_context,
        show_progress=True,
    )
    print("初始化本地向量完成")

collection_name: 'hybrid_nodes' 已存在
加载向量完成


In [ ]:
# try:
#     hybrid_index.storage_context.persist(persist_dir)
#     print("持久化完成")
# except Exception as e:
#     print(e)

持久化完成


In [9]:
import jieba

stop_words_path = project_root / "readai/utils/baidu_stopwords.txt"
custom_tokenizer = jieba.Tokenizer()
custom_tokenizer.load_userdict(str(stop_words_path))
# 加载stop_words为字符串列表
with open(stop_words_path) as f:
    stop_words = f.readlines()

stop_words = set(stop_words)
print(len(stop_words))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\THINKT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.448 seconds.
Prefix dict has been built successfully.


1396


In [10]:
from enum import Enum


class FUSION_MODES(str, Enum):
    """Enum for different fusion modes."""

    RECIPROCAL_RANK = "reciprocal_rerank"  # apply reciprocal rank fusion
    RELATIVE_SCORE = "relative_score"  # apply relative score fusion
    DIST_BASED_SCORE = "dist_based_score"  # apply distance-based score fusion
    SIMPLE = "simple"  # simple re-ordering of results based on original scores

### 构建测试集

In [41]:
QA_GENERATE_EASY = """\
You are an assistant helping users read Chinese books.
Based on the context below, generate {num_questions_per_chunk} **easy** questions that test basic understanding or facts.

Context:
---------------------
{context_str}
---------------------

Rules:
- Questions must be written in **Chinese**.
- Do not include answers or question numbers.
- Each question must be concise (<= 20 characters).
- Only ask about **explicit content** from the context (e.g., names, definitions, facts).
- Avoid "这", "那", "根据" and similar vague words.

Example question:
某理论的核心概念是?
"""

In [42]:
import random

# 筛选出hybrid_nodes中text长度>800的节点
print(len(hybrid_nodes))
hybrid_nodes_filtered = [node for node in hybrid_nodes if len(node.text) >= 800]
print("过滤后节点数", len(hybrid_nodes_filtered))

170
过滤后节点数 119


In [43]:
sample_size = 50
hybrid_nodes_filtered_sample = random.sample(hybrid_nodes_filtered, sample_size)

In [44]:
# 基于hybrid_nodes_filtered构建语料数据集,并保存到csv文件中,包含三列，一行为node_id，一行为content，一行为questions
import pandas as pd
from tqdm import tqdm

rows = []
for node in tqdm(hybrid_nodes_filtered_sample, desc="使用deepseek-v3构建QA数据集"):
    node_id = node.node_id
    content = node.text
    # 用content构建query，然后调用模型生成问题
    query = QA_GENERATE_EASY.format(context_str=content, num_questions_per_chunk=2)
    response = llm.complete(query)
    questions = str(response).strip().split("\n")
    questions = "#".join(questions)
    rows.append((node_id, content, questions))

# 将node_contents保存到csv文件中
df = pd.DataFrame(rows, columns=["node_id", "content", "questions"])
df.to_csv("./qa_dataset_v2.csv", index=False)

使用deepseek-v3构建QA数据集: 100%|██████████| 50/50 [03:57<00:00,  4.74s/it]


In [45]:
# 生成queries字典
# query_id通过str(uuid.uuid4())生成，作为key
# 列questions由于通过#连接的，需要分割成为两个问题，因此读取df每行的时候，会创建两个key：value对
# 同时更新relevant_docs字典，把query_id作为key,node_id作为value
import uuid
import json

corpus = {}
for index, row in df.iterrows():
    node_id = row["node_id"]
    content = row["content"]
    corpus[node_id] = content


queries = {}
relevant_docs = {}
for index, row in df.iterrows():
    # 两个不同的query_id
    query_id_q1 = str(uuid.uuid4())
    query_id_q2 = str(uuid.uuid4())
    questions = row["questions"].replace("##", "#").split("#")
    questions = [question.strip() for question in questions]  # 去除前后空格
    queries[query_id_q1] = questions[0]  # 问题1
    relevant_docs[query_id_q1] = [row["node_id"]]
    queries[query_id_q2] = questions[1]  # 问题2
    relevant_docs[query_id_q2] = [row["node_id"]]

test_qa_datasets_json = {
    "queries": queries,
    "corpus": corpus,
    "relevant_docs": relevant_docs,
}
with open("test_qa_datasets.json", "w") as f:
    json.dump(test_qa_datasets_json, f, indent=4, ensure_ascii=False)

In [18]:
eval_dataset_from_hybrid_nodes_50 = EmbeddingQAFinetuneDataset.from_json(
    "test_qa_datasets.json",
)

### 构建retriever测试

In [11]:
from llama_index.core.retrievers import RecursiveRetriever

from readai.components.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_defaults(
    index=hybrid_index,
    similarity_top_k=5,
    tokenizer=custom_tokenizer,
    stopwords=stop_words,
)

vector_retriever = hybrid_index.as_retriever(similarity_top_k=5)
small2big_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=hybrid_index.docstore.docs,
    verbose=True,
)

resource module not available on Windows


In [12]:
from llama_index.core.response.notebook_utils import display_source_node

In [22]:
sparse_nodes = bm25_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)
print(len(sparse_nodes))
for n in sparse_nodes:
    display_source_node(n, source_length=1500)

4


**Node ID:** cc224a0c-f6b7-4d8e-8586-b3f7ea182834<br>**Similarity:** 26.955977526355234<br>**Text:** 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** 4e46cffd-e5b8-4cb0-b886-8e8356908cd1<br>**Similarity:** 26.268064015672522<br>**Text:** 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。<br>

**Node ID:** dc4e4cc1-4e38-40a1-9265-848096ffb88d<br>**Similarity:** 26.214797279821816<br>**Text:** 会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。<br>

**Node ID:** 28bdd00d-d219-4193-9034-e1e5c602c15d<br>**Similarity:** 24.495895670924263<br>**Text:** ” 刚开始尝试请他人重述我们的话时，可能会觉得有点不自然。因为在日常交流中，我们很少会提出这样的请求。当我强调请他人重述我们的话是一种重要的能力时，人们通常担心有人会这样反应：“你觉得我是聋子吗？<br>

In [ ]:
node_with_score_example = sparse_nodes[0]
print(node_with_score_example.text)  # node的文本
print(node_with_score_example.score)  # node的分数

区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。
26.955977526355234


In [43]:
dense_nodes = small2big_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)
print(len(dense_nodes))
for n in dense_nodes:
    display_source_node(n, source_length=1500)

Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: ea01298b-1034-4b63-b2b0-dae899b25dce
Retrieving with query id ea01298b-1034-4b63-b2b0-dae899b25dce: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 3423a0b8-f692-4abe-b5f0-26f4bd41cc99
Retrieving with query id 3423a0b8-f692-4abe-b5f0-26f4bd41cc99: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 28686fee-f722-4232-85c6-13e1ec89927d
Retrieving with query id 28686fee-f722-4232-85c6-13e1ec89927d: 在日常交流中，我们如何区分客观的观察和主观的评价？
3


**Node ID:** ea01298b-1034-4b63-b2b0-dae899b25dce<br>**Similarity:** 0.6396604<br>**Text:** 当我指出这一点后，另一位老师响应道：“我知道他的意思了。校长的话太多！”这仍不是一个清晰的观察，而是对校长说多少话的评论。随后，第三位老师说：“他认为只有他想说的话是重要的。”我进而向他们解释，推断他人的想法和对他人行为的观察是两码事。随后，第四位老师大胆地说：“他总是想成为人前的焦点。”当我指出这也是推断时，两位老师不约而同地说道：“你的问题太难回答了！” 接着，我们一起拟了份清单，明确列出校长有哪些具体行为令他们感到不满，并确保不掺杂评论。例如，在全体教师会议上，校长会讲述他的童年和战争经历，有时导致会议超时20分钟。我问老师们是否曾经和校长沟通过他们的不满，他们说曾经试过，但都用具有评论意味的言辞向校长提出批评，而从未提及任何具体行为，例如校长在会议中讲述自己的故事。最后，老师们同意在与校长会谈时将这一点提出来。 会谈刚一开始，我便目睹了老师们所描述的情景。不论讨论的主题是什么，校长都会插话说：“想当年……”接着开始讲述他的童年或战争经历。我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** 3423a0b8-f692-4abe-b5f0-26f4bd41cc99<br>**Similarity:** 0.63307714<br>**Text:** ” “不是这里，”男子回答，“我把钥匙落在巷子里了。” 见到警察很困惑，男子急忙解释：“但这里的光线要亮得多呢。” 我们都对人生拥有渴望。然而我发现，主导我们的文化却可能带我们偏离了方向。为此，我开发了非暴力沟通。它作为一种训练注意力的方式，让我们可以把注意力放在真正要去的地方。我渴望的是，在生命中活出慈悲之心，让由衷的相互给予，在你我之间流动。 那份慈悲心，也是我所说的“由衷的给予”，在我的朋友鲁斯·贝本梅尔（Ruth Bebermeyer）的这段歌词中得到展现： 你取之于我，是对我最好的馈赠。 请你知道，这种给予带给我快乐。 我给予绝非冀盼你偿还， 只因我想活出对你的爱。 也许，你的欣然接受， 就是对我的最佳馈赠。 我无法将两者分开。 你施于我，我给予我的接受； 你取之于我，是对我最好的馈赠。 ——“馈赠”（1978）来自鲁斯·贝本梅尔 唱片集《馈赠》 每当我们想要丰盈他人的生命，由衷给予他人时，我们是出于心中油然而生的喜悦之情。给予者和接受者都会从中获益。当给予是由衷的，而非出于害怕、内疚、羞愧或是渴求回报，接受者才能心无挂碍地享受这份馈赠。与此同时，给予者因为看到自己为他人的幸福做出了贡献，也会收获更多的自我肯定。 我们使用非暴力沟通，并不要求沟通的对方也懂得它，甚至无须对我们保有善意。只要我们遵循非暴力沟通的原则，发自内心地给予，并尽我们所能让对方知道这是我们唯一的动机，他们也会加入我们，最终双方必然能以善意相待。当然，这一切不一定会很快发生，但我相信，只要我们坚持非暴力沟通的原则与方法，友善之花终将在你我心中盛开。 非暴力沟通过程 要达到让双方都能发自内心地相互给予，我们需将注意力聚焦在四个方面——它们是非暴力沟通模式的四个要素。 首先，我们观察实际上发生了什么。不论他人的言行是否有益于我们，我们只是去观察。要做到清晰表达所观察到的，我们的挑战在于不夹杂任何评判。不论喜欢与否，我们只是说出人们做了什么。<br>

**Node ID:** 28686fee-f722-4232-85c6-13e1ec89927d<br>**Similarity:** 0.629032<br>**Text:** 去观察，就像信仰一样重要。 ——弗雷德里克·布希纳（Frederick Buechner） 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 如果你想把事情搅乱， 我可以告诉你如何做到： 将我做的事情 和你的反应混为一谈。 当你见到做了一半的家务活， 可以告诉我你感到失望。 但说我不负责任， 绝无可能让我做得更多。 当我对你的表白说“不”， 请告诉我你感到伤心。 但说我冷酷无情， 并不能给你带来更多机会。 是的， 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 ——马歇尔·卢森堡博士 非暴力沟通的第一个要素是区分观察与评论。我们要清楚地观察有哪些所见、所闻和所触，正影响着我们幸福，而不夹杂任何评论。 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。如果我们在观察中夹杂着评论，人们便不那么容易真正听见我们想要表达的内容，反而会听到批评，甚至产生抗拒心理。 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。语义学家温戴尔·约翰逊（Wendell Johnson）指出，用静态的语言来表达或捕捉瞬息万变的现实世界，会带来许多问题。他说：“我们的语言年代久远，是有缺陷的工具。它反映着泛灵论的思想，倾向于谈论稳定与恒常，谈论相似性、常态和分类，神奇的转变、立竿见影的疗效、简化的问题以及终极解决方案。然而，我们尝试用语言符号化的这个世界，却包含了无穷无尽的过程、变化、维度、变数、关系、功能、互动、发展、学习、应对和复杂性。不断变化的世界与我们相对静态的语言形式并不匹配，我们的许多问题正是因此产生。” 我的同事鲁斯·贝本梅尔在这段歌词中对比了静态语言与动态语言，由此来展现评论与观察的区别： 我从未见过什么懒汉； 我见过的他，有时在白天睡觉， 在某个下雨的日子呆在家里。 但他不是个懒汉。<br>

In [27]:
dense_nodes_with_score_example = dense_nodes[0]
print(dense_nodes_with_score_example.text)
print(dense_nodes_with_score_example.score)

当我指出这一点后，另一位老师响应道：“我知道他的意思了。校长的话太多！”这仍不是一个清晰的观察，而是对校长说多少话的评论。随后，第三位老师说：“他认为只有他想说的话是重要的。”我进而向他们解释，推断他人的想法和对他人行为的观察是两码事。随后，第四位老师大胆地说：“他总是想成为人前的焦点。”当我指出这也是推断时，两位老师不约而同地说道：“你的问题太难回答了！” 接着，我们一起拟了份清单，明确列出校长有哪些具体行为令他们感到不满，并确保不掺杂评论。例如，在全体教师会议上，校长会讲述他的童年和战争经历，有时导致会议超时20分钟。我问老师们是否曾经和校长沟通过他们的不满，他们说曾经试过，但都用具有评论意味的言辞向校长提出批评，而从未提及任何具体行为，例如校长在会议中讲述自己的故事。最后，老师们同意在与校长会谈时将这一点提出来。 会谈刚一开始，我便目睹了老师们所描述的情景。不论讨论的主题是什么，校长都会插话说：“想当年……”接着开始讲述他的童年或战争经历。我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。
0.6396604


In [13]:
from llama_index.core.retrievers import QueryFusionRetriever

fusion_retriever = QueryFusionRetriever(
    retrievers=[small2big_retriever, bm25_retriever],
    mode=FUSION_MODES.RECIPROCAL_RANK,
    verbose=True,
    similarity_top_k=5,
    llm=llm,
    num_queries=1,  # 不需要查询扩展,查询重写
)

In [16]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=fusion_retriever
)

In [19]:
eval_results_hybrid_nodes_50 = await retriever_evaluator.aevaluate_dataset(
    eval_dataset_from_hybrid_nodes_50, show_progress=True
)

  0%|          | 0/100 [00:00<?, ?it/s]

Retrieving with query id None: 调解过程中需要记录什么？


2025-04-10 23:01:22.188 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-10 23:01:22.248 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-10 23:01:22.328 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  1%|          | 1/100 [00:01<01:55,  1.17s/it]2025-04-10 23:01:22.385 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: d10d3dd5-4d70-49a6-ab03-5618b8fda079
Retrieving with query id d10d3dd5-4d70-49a6-ab03-5618b8fda079: 调解过程中需要记录什么？
Retrieved node with id, entering: 8ed32533-b275-4200-959b-3e985d537889
Retrieving with query id 8ed32533-b275-4200-959b-3e985d537889: 调解过程中需要记录什么？
Retrieved node with id, entering: 7f282a20-f873-4037-a0bf-f3aa3e088b28
Retrieving with query id 7f282a20-f873-4037-a0bf-f3aa3e088b28: 调解过程中需要记录什么？
Retrieving with query id None: 调解人如何推动对话进程？
Retrieved node with id, entering: d10d3dd5-4d70-49a6-ab03-5618b8fda079
Retrieving with query id d10d3dd5-4d70-49a6-ab03-5618b8fda079: 调解人如何推动对话进程？
Retrieving with query id None: 约翰为什么沉默不语？
Retrieved node with id, entering: 2fc1d0a9-dd61-4ed4-86f7-81ac8ef40516
Retrieving with query id 2fc1d0a9-dd61-4ed4-86f7-81ac8ef40516: 约翰为什么沉默不语？
Retrieved node with id, entering: 3f1da84d-975b-4ac0-9e19-6c5695293de3
Retrieving with query id 3f1da84d-975b-4ac0-9e19-6c5695293de3: 约翰为什么沉默不语？
Retrieving with query id None: 布莱特拿了

2025-04-10 23:01:22.490 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
  3%|▎         | 3/100 [00:01<00:35,  2.72it/s]2025-04-10 23:01:22.557 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


...

Retrieved node with id, entering: eb6b259a-e250-429f-b068-6277fa472750
Retrieving with query id eb6b259a-e250-429f-b068-6277fa472750: 作者如何改变开车时的心态？
Retrieving with query id None: 女士最初为何坚持做饭？
Retrieved node with id, entering: 8a4827fa-68cf-460b-919b-b25c5b9ee9d3
Retrieving with query id 8a4827fa-68cf-460b-919b-b25c5b9ee9d3: 女士最初为何坚持做饭？
Retrieved node with id, entering: 873efb03-d371-44cc-9383-2492942f3469
Retrieving with query id 873efb03-d371-44cc-9383-2492942f3469: 女士最初为何坚持做饭？
Retrieved node with id, entering: 46487957-5414-4a35-9cdb-f5d0f17f1bfc
Retrieving with query id 46487957-5414-4a35-9cdb-f5d0f17f1bfc: 女士最初为何坚持做饭？
Retrieving with query id None: 儿子们对母亲的决定有何反应？
Retrieved node with id, entering: 8a4827fa-68cf-460b-919b-b25c5b9ee9d3
Retrieving with query id 8a4827fa-68cf-460b-919b-b25c5b9ee9d3: 儿子们对母亲的决定有何反应？
Retrieved node with id, entering: 7b175426-8398-459e-8307-be8f165b2a6c
Retrieving with query id 7b175426-8398-459e-8307-be8f165b2a6c: 儿子们对母亲的决定有何反应？
Retrieved node with id, ent

 93%|█████████▎| 93/100 [00:06<00:00, 16.22it/s]2025-04-10 23:01:27.556 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-10 23:01:27.620 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
 95%|█████████▌| 95/100 [00:06<00:00, 16.43it/s]2025-04-10 23:01:27.671 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 校长用什么方式训斥大孩子？
Retrieved node with id, entering: d6471822-d837-4a9f-ad2d-ef25068b4eb3
Retrieving with query id d6471822-d837-4a9f-ad2d-ef25068b4eb3: 校长用什么方式训斥大孩子？
Retrieved node with id, entering: adf4c544-1698-43c6-9807-e07c8c4bb007
Retrieving with query id adf4c544-1698-43c6-9807-e07c8c4bb007: 校长用什么方式训斥大孩子？
Retrieved node with id, entering: 7ede71a1-2630-40af-bd0d-9b059e22016b
Retrieving with query id 7ede71a1-2630-40af-bd0d-9b059e22016b: 校长用什么方式训斥大孩子？
Retrieving with query id None: 谈话中多少人看向了说话者？
Retrieved node with id, entering: 2dba6dd1-44f7-483f-9844-1a2d67ba3c86
Retrieving with query id 2dba6dd1-44f7-483f-9844-1a2d67ba3c86: 谈话中多少人看向了说话者？
Retrieved node with id, entering: 7deab73b-4937-4215-be40-21ea5d2cde73
Retrieving with query id 7deab73b-4937-4215-be40-21ea5d2cde73: 谈话中多少人看向了说话者？
Retrieved node with id, entering: ea01298b-1034-4b63-b2b0-dae899b25dce
Retrieving with query id ea01298b-1034-4b63-b2b0-dae899b25dce: 谈话中多少人看向了说话者？
Retrieved node with id

2025-04-10 23:01:27.718 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-10 23:01:27.808 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
100%|██████████| 100/100 [00:06<00:00, 15.00it/s]

Retrieved node with id, entering: ceb2fcca-64e2-46a9-ac95-ccb2b6a6e488
Retrieving with query id ceb2fcca-64e2-46a9-ac95-ccb2b6a6e488: 作者从哪位老师身上学到东西？
Retrieved node with id, entering: 3382f2dd-2a25-4b80-a16c-06e1a55ad6bb
Retrieving with query id 3382f2dd-2a25-4b80-a16c-06e1a55ad6bb: 作者从哪位老师身上学到东西？
Retrieved node with id, entering: ea01298b-1034-4b63-b2b0-dae899b25dce
Retrieving with query id ea01298b-1034-4b63-b2b0-dae899b25dce: 作者从哪位老师身上学到东西？
Retrieved node with id, entering: 36efd65b-9730-4f80-8122-df1e9647aab1
Retrieving with query id 36efd65b-9730-4f80-8122-df1e9647aab1: 作者从哪位老师身上学到东西？
Retrieving with query id None: 舅舅照顾谁时总是面带微笑？
Retrieved node with id, entering: 8fca914a-47f8-45ae-9231-c2074b567b0f
Retrieving with query id 8fca914a-47f8-45ae-9231-c2074b567b0f: 舅舅照顾谁时总是面带微笑？
Retrieved node with id, entering: 49a38ab4-bb6c-46c0-a06e-568fe4ec8c9d
Retrieving with query id 49a38ab4-bb6c-46c0-a06e-568fe4ec8c9d: 舅舅照顾谁时总是面带微笑？


In [20]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""
    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)
    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [21]:
display_results("hybrid_nodes_50", eval_results_hybrid_nodes_50)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.6,0.505667,0.122833,0.6,0.505667,0.529046


### 结合reranker model测试

In [69]:
from readai.components.rerank_retriever import MyReRankRetriever

In [ ]:
local_model_path = os.path.join(project_root, "model/embeddings/bge-reranker-v2-m3")
rerank_retriever = MyReRankRetriever(
    retrievers=[bm25_retriever, small2big_retriever],
    similarity_top_k=5,
    use_fp16=False,
    devices=[0],
    batch_size=32,
    verbose=True,
    model_name_or_path=local_model_path,
)

In [53]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
rerank_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=rerank_retriever
)

In [ ]:
reranled_nodes = rerank_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)

In [55]:
for node in reranled_nodes:
    display_source_node(node, source_length=1000)

**Node ID:** 28686fee-f722-4232-85c6-13e1ec89927d<br>**Similarity:** 1.1604560613632202<br>**Text:** 去观察，就像信仰一样重要。 ——弗雷德里克·布希纳（Frederick Buechner） 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 如果你想把事情搅乱， 我可以告诉你如何做到： 将我做的事情 和你的反应混为一谈。 当你见到做了一半的家务活， 可以告诉我你感到失望。 但说我不负责任， 绝无可能让我做得更多。 当我对你的表白说“不”， 请告诉我你感到伤心。 但说我冷酷无情， 并不能给你带来更多机会。 是的， 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 ——马歇尔·卢森堡博士 非暴力沟通的第一个要素是区分观察与评论。我们要清楚地观察有哪些所见、所闻和所触，正影响着我们幸福，而不夹杂任何评论。 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。如果我们在观察中夹杂着评论，人们便不那么容易真正听见我们想要表达的内容，反而会听到批评，甚至产生抗拒心理。 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。语义学家温戴尔·约翰逊（Wendell Johnson）指出，用静态的语言来表达或捕捉瞬息万变的现实世界，会带来许多问题。他说：“我们的语言年代久远，是有缺陷的工具。它反映着泛灵论的思想，倾向于谈论稳定与恒常，谈论相似性、常态和分类，神奇的转变、立竿见影的疗效、简化的问题以及终极解决方案。然而，我们尝试用语言符号化的这个世界，却包含了无穷无尽的过程、变化、维度、变数、关系、功能、互动、发展、学习、应对和复杂性。不断变化的世界与我们相对静态的语言形式并不匹配，我们的许多问题正是因此产生。” 我的同事鲁斯·贝本梅尔在这段歌词中对比了静态语言与动态语言，由此来展现评论与观察的区别： 我从未见过什么懒汉； 我见过的他，有时在白天睡觉， 在某个下雨的日子呆在家里。 但他不是个懒汉。<br>

**Node ID:** 4e46cffd-e5b8-4cb0-b886-8e8356908cd1<br>**Similarity:** -0.2287135273218155<br>**Text:** 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。<br>

**Node ID:** cc224a0c-f6b7-4d8e-8586-b3f7ea182834<br>**Similarity:** -1.8991965055465698<br>**Text:** 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** dc4e4cc1-4e38-40a1-9265-848096ffb88d<br>**Similarity:** -2.291855573654175<br>**Text:** 会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。<br>

**Node ID:** ea01298b-1034-4b63-b2b0-dae899b25dce<br>**Similarity:** -3.1947731971740723<br>**Text:** 当我指出这一点后，另一位老师响应道：“我知道他的意思了。校长的话太多！”这仍不是一个清晰的观察，而是对校长说多少话的评论。随后，第三位老师说：“他认为只有他想说的话是重要的。”我进而向他们解释，推断他人的想法和对他人行为的观察是两码事。随后，第四位老师大胆地说：“他总是想成为人前的焦点。”当我指出这也是推断时，两位老师不约而同地说道：“你的问题太难回答了！” 接着，我们一起拟了份清单，明确列出校长有哪些具体行为令他们感到不满，并确保不掺杂评论。例如，在全体教师会议上，校长会讲述他的童年和战争经历，有时导致会议超时20分钟。我问老师们是否曾经和校长沟通过他们的不满，他们说曾经试过，但都用具有评论意味的言辞向校长提出批评，而从未提及任何具体行为，例如校长在会议中讲述自己的故事。最后，老师们同意在与校长会谈时将这一点提出来。 会谈刚一开始，我便目睹了老师们所描述的情景。不论讨论的主题是什么，校长都会插话说：“想当年……”接着开始讲述他的童年或战争经历。我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

In [ ]:
eval_rerank_result = await rerank_evaluator.aevaluate_dataset(
    eval_dataset_from_hybrid_nodes_50, show_progress=True
)

In [57]:
display_results("hybrid_nodes_50_rerank", eval_rerank_result)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50_rerank,0.61,0.349333,0.124833,0.61,0.349333,0.414313


测试使用flagreranker

In [ ]:
from FlagEmbedding import FlagReranker

reranker = FlagReranker(
    model_name_or_path=local_model_path, use_fp16=True, devices=[0], batch_size=32
)

In [36]:
sentence_pairs = [
    ("机器学习是什么？", "机器学习是一种让计算机根据数据自动改进的技术。"),
    (
        "人工智能可以应用在哪些领域？",
        "人工智能可以广泛应用于医疗、金融、教育、制造业等多个领域。",
    ),
    (
        "深度学习与机器学习有什么区别？",
        "深度学习是机器学习的一种方法，专注于使用神经网络从大量数据中学习。",
    ),
    (
        "什么是自然语言处理？",
        "自然语言处理是让计算机能够理解、生成和与人类语言交互的技术。",
    ),
    (
        "大数据分析的核心目标是什么？",
        "大数据分析的核心目标是从大量数据中提取有价值的信息，支持决策和预测。",
    ),
]
reranker.compute_score(sentence_pairs)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[7.1328125, 7.51171875, 3.732421875, 8.203125, 8.796875]